<a href="https://colab.research.google.com/github/Koumi94/ALIOU/blob/main/yapayzeka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import IPython.display as disp
!pip install osmnx
!apt install libspatialindex-dev
import osmnx as ox
import networkx as nx
from datetime import timedelta
from shapely.geometry import Polygon, LineString, Point
import itertools as iter
import math
%matplotlib inline
ox.config(log_console=True)
ox.__version__


In [35]:

import pandas as pd
# Import the ML library
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV,PassiveAggressiveRegressor,LinearRegression,ARDRegression,HuberRegressor,ElasticNetCV,OrthogonalMatchingPursuit,ElasticNet,RidgeClassifier,RidgeCV,SGDClassifier,BayesianRidge, Perceptron,RANSACRegressor,LassoLars,Lasso,LassoCV,PassiveAggressiveClassifier
from sklearn.metrics import balanced_accuracy_score,accuracy_score,mean_squared_error
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier,RandomForestClassifier,VotingClassifier,AdaBoostClassifier
import numpy as np
import time
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('covid_19_Guinea.csv',sep=';')
#print(data)

data.drop('Sno',axis =1)
data.rename(columns ={'State/UnionTerritory' : 'City', 'ConfirmedGuineanNational': 'GuineanConfirmed'}, inplace = True)
total_Cases_ByCity = data.groupby('City')[['Deaths','Cured','GuineanConfirmed', 'Confirmed']].sum()
total_Cases_ByCity.head(10)
data.head()
data = data[['City','Date' ,'GuineanConfirmed', 'ConfirmedForeignNational', 'Deaths','Cured', 'Confirmed']]
 
#print
data['City'].nunique()


In [ ]:
# Resetting Date to  Datetime object and making it an index of dataframe

data = data.fillna('NaN')
pivot_value=pd.pivot_table(data,columns='City',
                     index='Date',
                     values='Confirmed',
                     aggfunc=np.sum)

pivot_Deaths=pd.pivot_table(data,
                  columns='City',
                  index='Date',
                  values='Deaths',
                  aggfunc=np.sum)
pivot_Cured=pd.pivot_table(data,
                  columns='City',
                  index='Date',
                  values='Cured',
                  aggfunc=np.sum)



#print(pivot_Deaths)
city_list=[]
value_list=[]
deaths_list=[]
cured_list=[]

for country in list(pivot_value.columns):
    city_list.append(country)
    value_list.append(pivot_value[country].max())
    deaths_list.append(pivot_Deaths[country].max())
    cured_list.append(pivot_Cured[country].max())
  
new_dict={'Cities':city_list,'Confirmed':value_list,'Deaths':deaths_list , 'Cured':cured_list }

df=pd.DataFrame.from_dict(new_dict)
df.set_index('Cities',inplace=True)
df



In [ ]:
sortdf_confirmed = df.sort_values(by=['Confirmed'],ascending=False)
sortdf_Deaths = df.sort_values(by=['Deaths'],ascending=False)
sortdf_Cured = df.sort_values(by=['Cured'],ascending=False)
top10_conf = sortdf_confirmed.head(10)
top10_deat = sortdf_Deaths.head(10)
top10_cured = sortdf_Deaths.head(10)
print('     Top 10 of Confirmed cases: ' )
print(top10_conf)
print('     Top 10 of Deaths: ' )
print(top10_deat)
print('     Top 10 of Cured: ' )
print(top10_cured)



In [ ]:
import plotly.express as px
fig = px.bar(top10_conf, x=top10_conf.index, y='Confirmed', 
             labels={'x':'Country'}, color="Confirmed", 
             color_continuous_scale=px.colors.sequential.Rainbow_r)
fig.update_layout(title_text='Top 10 Confirmed COVID-19 cases by country')
fig.show()

fig = px.bar(top10_cured, x=top10_conf.index, y='Cured', 
             labels={'x':'Country'}, color="Cured", 
             color_continuous_scale=px.colors.sequential.Rainbow_r)
fig.update_layout(title_text='Top 10 Cured COVID-19 cases by country')
fig.show()

fig = px.bar(top10_conf, x=top10_deat.index, y='Deaths', 
             labels={'x':'Country'}, color="Deaths", 
             color_continuous_scale=px.colors.sequential.Rainbow_r)
fig.update_layout(title_text='Top 10 Deaths COVID-19 cases by country')
fig.show()


In [ ]:

def Lsite_optimization(vacc, vill, 
                       L = 3, graph_area = ("San Juan, Batangas, Philippines"),
                        distance = "road", return_distances = False, enumerative = False):
  '''
  @vacc =   Vaccination centers dataframe
  @villages =  Village centers dataframe
  @L =  number of vaccination sites to be used for optimization
  graph_area = string of the region name
  distance = string of the distance type, can be "road", "euclidean", or 'time
  return_distances = If true, the function will return the distances matrix between each village center and each vaccination site
  enumerative = If true, every possible combination of vaccination centers will be iterated on. 
                 If false, genetic algorithms will be used for speed.  
  '''
  
  TI = vill.infected.sum()
  TP = vill.population.sum() 
  vill["weight"] = vill.infected/TI + vill.population/TP

  ##  Automated N-site Optimization!!! ================================================================

  ## Select Open Street Map code for the given province 

  graph_area = graph_area

  # Create the graph of the area from OSM data. It will download the data and create the graph
  G = ox.graph_from_place(graph_area, network_type='drive')
  G = ox.add_edge_speeds(G)
  G = ox.add_edge_travel_times(G)


  ## Phase 1: N-site  Distance Matrix------------------------------------------------------------------
  index = vacc.Name
  columns = vill.Village_name
  df_distances = pd.DataFrame(index=index, columns = columns)
  
  for i in vacc.index:
    for j in vill.index:
      if distance == "road":
        #Get pure road distance
        origin_node = ox.distance.nearest_nodes(G, Y = vill.iloc[j].latitude, X = vill.iloc[j].longitude)
        destination_node = ox.distance.nearest_nodes(G, Y = vacc.iloc[i].latitude, X = vacc.iloc[i].longitude)
        
        df_distances.iloc[i,j] = nx.shortest_path_length(G, 
                                            origin_node, destination_node, weight='length')
      if distance == "euclidean":
        df_distances.iloc[i,j] = ox.distance.euclidean_dist_vec(
          vacc.iloc[i].latitude, vacc.iloc[i].longitude, vill.iloc[j].latitude, vacc.iloc[j].longitude)

      if distance == "time":
        #Get pure road distance
        origin_node = ox.distance.nearest_nodes(G, Y = vill.iloc[j].latitude, X = vill.iloc[j].longitude)
        destination_node = ox.distance.nearest_nodes(G, Y = vacc.iloc[i].latitude, X = vacc.iloc[i].longitude)
        
        df_distances.iloc[i,j] = nx.shortest_path_length(G, 
                                            origin_node, destination_node, weight='travel_time')


  index1 = list(iter.combinations(df_distances.index, L))
  master = pd.DataFrame(index=index1, columns = vill.Village_name)

  print("phase 1 complete,  distance matrix computed")

  #Phase 2 ---------------------------------------------------------------------------

  if enumerative is True:
    for i, name in enumerate(list(iter.combinations(df_distances.index, L))):
        #print(i)
        for j in vill.index:
          # Get weights
          weight = vill["weight"][j]
          sads = [None]*L
          for k in np.arange(0,L):
            sads[k] = df_distances.loc[name[k]][j]
          master.iloc[i,j] = weight*min(sads)
    print("phase 2 complete, optimization finished")

    ## Display Best Vaccination centers! 
    results_Lsite = pd.DataFrame(master.sum(axis=1))
    results_Lsite.columns = ["Cost"]
    if return_distances is True:
      return results_Lsite.sort_values(by=['Cost'], ascending=True), df_distances
    else:
      return results_Lsite.sort_values(by=['Cost'], ascending=True) 


  else:

    ## Combinatorial Genetic Algorithm Optimization 
    varbound=np.array([[0,vacc.shape[0]-1]]*L)
    
    master = pd.DataFrame(0, index=[0], columns=vill.Village_name)
    def f(x):
      x = x.tolist()
      for j in vill.index:
        # Get weights
        weight = vill["weight"][j]
        sads = [None]*L
        for k in np.arange(0,L):     
          sads[k] = df_distances.iloc[math.ceil(x[k]),j]
        master.iloc[0,j] = weight*min(sads)
      fitness = master.sum(axis=1)[0]
      return fitness
    

    algorithm_param = {'max_num_iteration': 300*L,
                      'population_size': 20*(L**2),
                      'mutation_probability':0.1,
                      'elit_ratio': 0.01,
                      'crossover_probability': 0.5,
                      'parents_portion': 0.3,
                      'crossover_type':'uniform',
                      'max_iteration_without_improv': 50}

    
    model = ga(function= f, dimension= L, variable_type='int', variable_boundaries=varbound, 
            algorithm_parameters=algorithm_param)

    print(model.run())
    

    ## Display Best Vaccination centers! 
    results_Lsite =  [int(i) for i in model.output_dict['variable'].tolist()]
    results_Lsite = tuple(df_distances.index[results_Lsite].tolist())
    wew = {'Cost': model.output_dict['function']}
    results_Lsite = pd.DataFrame(wew, index = [results_Lsite])
    
    if return_distances is True:
      return results_Lsite, df_distances
    else:
      return results_Lsite

def site_distribution(results, df_distances):

  ## results = results from the nsite_optimization function
  ## df_distances = distance matrix from the nsite_optimization function
    solution = df_distances.loc[[a for a in results.index[0]]]
    index = df_distances.columns
    columns = ["vaccination_center", "distance"]
    assignment  = pd.DataFrame(index=index, columns = columns)
    for i, name in enumerate(df_distances.columns):
      assignment.loc[name, "vaccination_center"] = solution.loc[:, [name]].sort_values(by=[name], ascending= True).index[0]
      assignment.loc[name, "distance"] = solution.loc[:, [name]].sort_values(by=[name], ascending=True).iloc[0, 0]
    
    distribution = assignment["vaccination_center"].value_counts()
    return assignment, distribution



In [40]:
def site_extraction(place = "San Juan, Batangas, Philippines", 
                    tags = { "amenity": ["hospital", "school", "kindergarten", "college", "university"]}):
  
  ## Automated extraction of schools and hospitals

    tags = tags
    place = place
    # get all building footprints in some neighborhood
    df = ox.geometries_from_place(place, tags)
    df["latlong"] = df.geometry.centroid
    df.reset_index(level=0, inplace=True)
    #df =df[["name", "latlong"]]
    return df
def Lsite_optimization(vacc, vill, 
                       L = 3, graph_area = ("San Juan, Batangas, Philippines"),
                        distance = "road", return_distances = False, enumerative = False):
  '''
  @vacc =   Vaccination centers dataframe
  @villages =  Village centers dataframe
  @L =  number of vaccination sites to be used for optimization
  graph_area = string of the region name
  distance = string of the distance type, can be "road", "euclidean", or 'time
  return_distances = If true, the function will return the distances matrix between each village center and each vaccination site
  enumerative = If true, every possible combination of vaccination centers will be iterated on. 
                 If false, genetic algorithms will be used for speed.  
  '''
  
  TI = vill.infected.sum()
  TP = vill.population.sum() 
  vill["weight"] = vill.infected/TI + vill.population/TP
  ##  Automated N-site Optimization!!! ================================================================

  ## Select Open Street Map code for the given province 

  graph_area = graph_area

  # Create the graph of the area from OSM data. It will download the data and create the graph
  G = ox.graph_from_place(graph_area, network_type='drive')
  G = ox.add_edge_speeds(G)
  G = ox.add_edge_travel_times(G)


  ## Phase 1: N-site  Distance Matrix------------------------------------------------------------------
  index = vacc.Name
  columns = vill.Village_name
  df_distances = pd.DataFrame(index=index, columns = columns)
  
  for i in vacc.index:
    for j in vill.index:
      if distance == "road":
        #Get pure road distance
        origin_node = ox.distance.nearest_nodes(G, Y = vill.iloc[j].latitude, X = vill.iloc[j].longitude)
        destination_node = ox.distance.nearest_nodes(G, Y = vacc.iloc[i].latitude, X = vacc.iloc[i].longitude)
        
        df_distances.iloc[i,j] = nx.shortest_path_length(G, 
                                            origin_node, destination_node, weight='length')
      if distance == "euclidean":
        df_distances.iloc[i,j] = ox.distance.euclidean_dist_vec(
          vacc.iloc[i].latitude, vacc.iloc[i].longitude, vill.iloc[j].latitude, vacc.iloc[j].longitude)
      if distance == "time":
        #Get pure road distance
        origin_node = ox.distance.nearest_nodes(G, Y = vill.iloc[j].latitude, X = vill.iloc[j].longitude)
        destination_node = ox.distance.nearest_nodes(G, Y = vacc.iloc[i].latitude, X = vacc.iloc[i].longitude)
        
        df_distances.iloc[i,j] = nx.shortest_path_length(G, 
                                            origin_node, destination_node, weight='travel_time')


  index1 = list(iter.combinations(df_distances.index, L))
  master = pd.DataFrame(index=index1, columns = vill.Village_name)

  print("phase 1 complete,  distance matrix computed")
 #Phase 2 ---------------------------------------------------------------------------

  if enumerative is True:
    for i, name in enumerate(list(iter.combinations(df_distances.index, L))):
        #print(i)
        for j in vill.index:
          # Get weights
          weight = vill["weight"][j]
          sads = [None]*L
          for k in np.arange(0,L):
            sads[k] = df_distances.loc[name[k]][j]
          master.iloc[i,j] = weight*min(sads)
    print("phase 2 complete, optimization finished")

    ## Display Best Vaccination centers! 
    results_Lsite = pd.DataFrame(master.sum(axis=1))
    results_Lsite.columns = ["Cost"]
    if return_distances is True:
      return results_Lsite.sort_values(by=['Cost'], ascending=True), df_distances
    else:
      return results_Lsite.sort_values(by=['Cost'], ascending=True) 


  else:

    ## Combinatorial Genetic Algorithm Optimization 
    varbound=np.array([[0,vacc.shape[0]-1]]*L)
    
    master = pd.DataFrame(0, index=[0], columns=vill.Village_name)
    def f(x):
      x = x.tolist()
      for j in vill.index:
        # Get weights
        weight = vill["weight"][j]
        sads = [None]*L
        for k in np.arange(0,L):     
          sads[k] = df_distances.iloc[math.ceil(x[k]),j]
        master.iloc[0,j] = weight*min(sads)
      fitness = master.sum(axis=1)[0]
      return fitness
    

    algorithm_param = {'max_num_iteration': 300*L,
                      'population_size': 20*(L**2),
                      'mutation_probability':0.1,
                      'elit_ratio': 0.01,
                      'crossover_probability': 0.5,
                      'parents_portion': 0.3,
                      'crossover_type':'uniform',
                      'max_iteration_without_improv': 50}

    
    model = ga(function= f, dimension= L, variable_type='int', variable_boundaries=varbound, 
            algorithm_parameters=algorithm_param)

    print(model.run())
     ## Display Best Vaccination centers! 
    results_Lsite =  [int(i) for i in model.output_dict['variable'].tolist()]
    results_Lsite = tuple(df_distances.index[results_Lsite].tolist())
    wew = {'Cost': model.output_dict['function']}
    results_Lsite = pd.DataFrame(wew, index = [results_Lsite])
    
    if return_distances is True:
      return results_Lsite, df_distances
    else:
      return results_Lsite

def site_distribution(results, df_distances):

  ## results = results from the nsite_optimization function
  ## df_distances = distance matrix from the nsite_optimization function
    solution = df_distances.loc[[a for a in results.index[0]]]
    index = df_distances.columns
    columns = ["vaccination_center", "distance"]
    assignment  = pd.DataFrame(index=index, columns = columns)
    for i, name in enumerate(df_distances.columns):
      assignment.loc[name, "vaccination_center"] = solution.loc[:, [name]].sort_values(by=[name], ascending= True).index[0]
      assignment.loc[name, "distance"] = solution.loc[:, [name]].sort_values(by=[name], ascending=True).iloc[0, 0]
    
    distribution = assignment["vaccination_center"].value_counts()
    return assignment, distribution
  
def optimal_sites(vaccination_centers_df, villages_df,
                       L = 3,  graph_area = ("San Juan, Batangas, Philippines"), return_ranking = False, enumerative = False,
                        distance = "road", return_distances = False, plot = True):
  '''
  PARAMS: 
  @vaccinatiOn_centers_df =   Vaccination centers dataframe
  @villages_df =  Village centers dataframe
  @L =  number of vaccination sites to be used for optimization
  graph_area = string of the region name
  distance = string of the distance type, can be "road", "euclidean", or 'time
  return_distances = If true, the function will return the distances matrix between each village center and each vaccination site
  enumerative = If true, every possible combination of vaccination centers will be iterated on. 
                 If false, genetic algorithms will be used for speed.  
  return_ranking = If true, then the results from the enumeration will be the output. This can only be done when enumerative = True.
  plot = If true then geospatial plot is shown
  '''
  vacc = vaccination_centers_df
  vill = villages_df
  TI = vill.infected.sum()
  TP = vill.population.sum() 
  vill["weight"] = vill.infected/TI + vill.population/TP
  
  result_nsites, df_distances = Lsite_optimization(vacc = vacc, vill = vill, L = L, graph_area = graph_area,
                                                   distance = distance, return_distances = True, enumerative = enumerative)
  assignment_nsite, distribution_nsite = site_distribution(results =  result_nsites, df_distances = df_distances)


  ## Plotting!!! =================================================================================================

  #Create the graph of the area from OSM data. 
  G = ox.graph_from_place(graph_area, network_type='drive')
  G = ox.add_edge_speeds(G)
  G = ox.add_edge_travel_times(G)

  ## Get coordinates of optimal sites
  optimal_sites = list(assignment_nsite.vaccination_center.unique())
  optimal_sites_loc = vacc[vacc.Name.isin(optimal_sites)] 
  optimal_sites_loc["colors"] = optimal_sites_loc["Name"].astype('category').cat.codes
  optimal_sites_loc

  # Get coordinates and assignments of barangays 
  wew = pd.merge(assignment_nsite, vill, on = "Village_name")
  wew["colors"] = wew["vaccination_center"].astype('category').cat.codes
  wew = wew.rename(columns={'vaccination_center': 'assigned_vaccination_center'})

  if plot is True:
      plt.figure(figsize=(25, 17), dpi=1500)
      fig, ax = ox.plot_graph(G, edge_color = None, show=False, close=False, bgcolor = "white",
                              edge_linewidth=1, node_size=1, figsize=(25, 20), dpi = 20000)

      ax.scatter(wew.longitude, wew.latitude, s= 100, c=wew["colors"], linewidths=1, edgecolors = "black", cmap='viridis')
      ax.scatter(optimal_sites_loc.longitude, optimal_sites_loc.latitude, s = 800, edgecolors = "black",
                c=optimal_sites_loc["colors"], linewidths=2, marker = "*")
      plt.show()
  if return_distances is True:
    if return_ranking is True:
      return assignment_nsite, df_distances, result_nsites
    else:
      return assignment_nsite, df_distances
  else:
    if return_ranking is True:
     return assignment_nsite, result_nsites
    else:
        return assignment_nsite

        

In [ ]:
## Load Data
vacc = pd.read_excel('Vaccination_Centers_Table.xlsx')
vill = pd.read_excel('Village_Centers_Table.xlsx')

#Set how many vaccination sites
L = 2

## Run enumerative algorithm and return village assignments, distance matrix, and ranking of all other suboptimal site combinations
assignment2, df_distances, results2 = optimal_sites(L = 2, vaccination_centers_df = vacc, villages_df = vill,
                        graph_area = ("San Juan, Batangas, Philippines"), return_ranking = True, enumerative = True,
                        distance = "road", return_distances = True, plot = True)

assignment2

In [ ]:
## Load Data
vacc = pd.read_excel('Vaccination_Centers_Table.xlsx')
vill = pd.read_excel('Village_Centers_Table.xlsx')



#Set how many vaccination sites
L = 4

## Run genetic algorithm and return village assignments for higher L 

assignment4 = optimal_sites(L = L, vaccination_centers_df = vacc, villages_df = vill,
                        graph_area = ("San Juan, Batangas, Philippines"))

assignment4

In [ ]:
covid_test = pd.read_csv('covid_vaccine_statewise.csv',sep=';')
print(covid_test)
X=covid_test.drop('Sessions', axis=0, inplace=True);
y=covid_test('Sessions');

#function defs:
def model_testing(model):
    start = time.time()
    #Split the data using test_train_split
    x_train,x_test,y_train,y_test = train_test_split(covid_test,test_size=0.5)
    #Fit the model using the data which was splitted
    model.fit(x_train,y_train)
    #Print Score of the model
    score = model.score(x_test,y_test)
    stop = time.time()
    timetaken = stop-start
    return (score,timetaken)
    #print(y_pred)
    
def plotGraph(keys, values, label, title):
    names = keys
    y_pos = np.arange(len(keys))
    data = values
    #plt.rcParams["figure.figsize"] = (15,7)
    plt.barh(y_pos, data, align='center', alpha=0.6)
    plt.yticks(y_pos, names)
    plt.xlabel(label)
    plt.title(title)
    plt.show()
    

alphas = [0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.7, 1]

model_dict = {
  
    "KNN": KNeighborsClassifier(n_neighbors = 7),
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "RandomForest": RandomForestClassifier(n_estimators=30, max_depth=9),
    "AdaBoostClassifier": AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=10,max_depth=4),n_estimators=10,learning_rate=0.6),
    "DecisionTreeClassifier" : DecisionTreeClassifier(),
    "LinearSVC" : LinearSVC(C = 1),
    "MultinomialNB": MultinomialNB(),
   
}
model_stats_score = {}
model_stats_timetaken = {}
for modelname in model_dict:
    #print(modelname)
    preds = []
    timetaken = []
    for i in range(0,20):
        model = model_dict[modelname]
        score,timeforfit=model_testing(model)
        timetaken.append(timeforfit)
        preds.append(score)
    prediction_mean = (sum(preds)/len(preds))
    model_stats_score[modelname] = prediction_mean
    timetaken_mean =  (sum(timetaken)/len(timetaken))
    model_stats_timetaken[modelname] = timetaken_mean
    print("average for {} : {} and time taken: {}".format(modelname,prediction_mean,timetaken_mean))